In [3]:
import pandas as pd
import zipfile
import os
from pykml import parser
from io import BytesIO
import xmltodict

def extract_coordinates(kml_folder):
    data = []
    for pm in kml_folder.findall('.//{http://www.opengis.net/kml/2.2}Placemark'):
        coords = pm.Point.coordinates.text.strip().split(',')
        name = pm.name.text.strip() if hasattr(pm, 'name') else "No Name"
        data.append({
            'Name': name,
            'Longitude': coords[0],
            'Latitude': coords[1],
            'Year': kml_folder.name.text.strip()
        })
    return data

def kmz_to_csv(kmz_filepath, csv_filepath):
    # Unzip KMZ to get to the KML file
    with zipfile.ZipFile(kmz_filepath, 'r') as zip_ref:
        kml_filename = [name for name in zip_ref.namelist() if name.endswith('.kml')][0]
        kml_data = zip_ref.read(kml_filename)

    # Parse KML file
    root = parser.fromstring(kml_data)

    # Extract Placemarks and respective data into a list
    all_data = []
    for folder in root.Document.Folder.findall('.//{http://www.opengis.net/kml/2.2}Folder'):
        all_data.extend(extract_coordinates(folder))

    # Convert list to a Pandas DataFrame
    df = pd.DataFrame(all_data)

    # Save DataFrame to CSV
    df.to_csv(csv_filepath, index=False)
    print(f"Data has been written to {csv_filepath}")




Data has been written to Septics_coordinates.csv


In [4]:
# Replace with your actual file paths
kmz_to_csv("Septics.kmz", "Septics_coordinates.csv")

Data has been written to Septics_coordinates.csv


In [5]:
def extract_coordinates(kml_folder):
    data = []
    for pm in kml_folder.findall('.//{http://www.opengis.net/kml/2.2}Placemark'):
        if hasattr(pm.Point, 'coordinates'):
            coords = pm.Point.coordinates.text.strip().split(',')
            # Convert from string to float and then to UTM
            longitude, latitude = float(coords[0]), float(coords[1])
            utm_coords = utm.from_latlon(latitude, longitude, force_zone_number=17)
            name = pm.name.text.strip() if hasattr(pm, 'name') else "No Name"
            data.append({
                'Name': name,
                'UTM Easting': utm_coords[0],
                'UTM Northing': utm_coords[1],
                'Year': kml_folder.name.text.strip()
            })
    return data


In [6]:
def extract_coordinates(kml_folder):
    data = []
    for pm in kml_folder.findall('.//{http://www.opengis.net/kml/2.2}Placemark'):
        if hasattr(pm.Point, 'coordinates'):
            coords = pm.Point.coordinates.text.strip().split(',')
            # Convert from string to float and then to UTM
            longitude, latitude = float(coords[0]), float(coords[1])
            utm_coords = utm.from_latlon(latitude, longitude, force_zone_number=17)
            name = pm.name.text.strip() if hasattr(pm, 'name') else "No Name"
            data.append({
                'Name': name,
                'UTM Easting': utm_coords[0],
                'UTM Northing': utm_coords[1],
                'Year': kml_folder.name.text.strip()
            })
    return data


In [7]:
import pandas as pd
import zipfile
from pykml import parser
import utm

def extract_coordinates(kml_folder):
    data = []
    for pm in kml_folder.findall('.//{http://www.opengis.net/kml/2.2}Placemark'):
        if hasattr(pm.Point, 'coordinates'):
            coords = pm.Point.coordinates.text.strip().split(',')
            # Convert from string to float and then to UTM
            longitude, latitude = float(coords[0]), float(coords[1])
            utm_coords = utm.from_latlon(latitude, longitude, force_zone_number=17)
            name = pm.name.text.strip() if hasattr(pm, 'name') else "No Name"
            data.append({
                'Name': name,
                'UTM Easting': utm_coords[0],
                'UTM Northing': utm_coords[1],
                'Year': kml_folder.name.text.strip()
            })
    return data

def kmz_to_csv(kmz_filepath, csv_filepath):
    # Unzip KMZ to get to the KML file
    with zipfile.ZipFile(kmz_filepath, 'r') as zip_ref:
        kml_filename = [name for name in zip_ref.namelist() if name.endswith('.kml')][0]
        kml_data = zip_ref.read(kml_filename)

    # Parse KML file
    root = parser.fromstring(kml_data)

    # Extract Placemarks and respective data into a list
    all_data = []
    for folder in root.Document.Folder.findall('.//{http://www.opengis.net/kml/2.2}Folder'):
        all_data.extend(extract_coordinates(folder))

    # Convert list to a Pandas DataFrame
    df = pd.DataFrame(all_data)

    # Save DataFrame to CSV
    df.to_csv(csv_filepath, index=False)
    print(f"Data has been written to {csv_filepath}")

# Replace with your actual file paths
kmz_to_csv("Septics.kmz", "Septics_coordinates.csv")


Data has been written to Septics_coordinates.csv


# Polygons

from pykml import parser
from lxml import etree
import pandas as pd


In [5]:
from pykml import parser
from lxml import etree
from shapely.geometry import Polygon


In [6]:
def extract_shapely_polygons(kml_file):
    # Open and parse the KML file
    with open(kml_file, 'r') as f:
        doc = parser.parse(f).getroot()

    # Dictionary to store year range and corresponding Shapely polygons
    polygons = {}

    # Iterate through folders (year ranges)
    for folder in doc.Document.Folder.Folder:
        year_range = folder.name.text
        polygons[year_range] = []

        # Iterate through Placemarks (polygons)
        for placemark in folder.Placemark:
            if hasattr(placemark, 'Polygon'):
                # Extract coordinates
                coords = placemark.Polygon.outerBoundaryIs.LinearRing.coordinates.text
                coord_list = [(float(c.split(',')[0]), float(c.split(',')[1])) for c in coords.strip().split(' ')]

                # Create Shapely Polygon and append to list
                polygon = Polygon(coord_list)
                polygons[year_range].append(polygon)

    return polygons


In [7]:
kml_file = 'Sanitary_Polygons.kml'  # Replace with your file path
polygon_shapes = extract_shapely_polygons(kml_file)

# Example: Print the polygons for a specific year range
year_range = '2011-2015'  # Replace with the actual year range in your KML file
for poly in polygon_shapes.get(year_range, []):
    print(poly)


POLYGON ((-80.08456895877413 44.46796204757844, -80.08363511311572 44.46814492562817, -80.08388947344989 44.46912844500715, -80.08438170831138 44.46903298594422, -80.08473196888549 44.4687671801755, -80.08456895877413 44.46796204757844))
POLYGON ((-80.07488730494181 44.46451841025498, -80.07328336466371 44.46530199545857, -80.07368590972227 44.46668532873691, -80.07543892305605 44.4664361702341, -80.07512234122404 44.46541067371765, -80.07488730494181 44.46451841025498))


In [9]:
polygon_shapes.keys()

dict_keys(['2006-2010', '2001-2005', '1996-2000', '2011-2015'])